In [4]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
from reprojection import *

In [5]:
files = sorted(glob.glob('YOUR PATH HERE/*'))

Reading the data

In [21]:
file1 = files[100]
file2 = files[120]

with fits.open(file1) as hdul:
    header1 = hdul[0].header
    data1 = hdul[0].data

with fits.open(file2) as hdul:
    header2 = hdul[0].header
    data2 = hdul[0].data

print(file1, file2)

/home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250508T092003_V202601302340_0545080504.fits.gz /home/ulyanov/data/solo/phi/polar/solo_L2_phi-fdt-blos_20250511T122002_V202601302340_0545110502.fits.gz


Showing the input data

In [7]:
def show_data(data, header, figsize=(10,10)):
    view = View.from_header(header)
    transform = ~view.to_spherical()

    grid = np.mgrid[-90:91:1,-180:180:1]
    grid, _ = transform(grid)
    grid = np.array(grid)

    meridians = grid[:,:,::15]
    parallels = np.transpose(grid[:,::15,:], (0,2,1))

    plt.figure(figsize=figsize)
    plt.imshow(data, origin='lower', cmap='seismic', vmin=-40, vmax=40)
    plt.plot(meridians[1], meridians[0], color='black', ls='--', lw=1, alpha=0.5)
    plt.plot(parallels[1], parallels[0], color='black', ls='--', lw=1, alpha=0.5)
    plt.tight_layout()

In [15]:
show_data(data1, header1)

In [9]:
show_data(data2, header2)

Reading the WCS information from header into the View class and calculating the transformation from local reference frame to Carrington coordinates

In [10]:
view1 = View.from_header(header1)
transform1 = view1.to_spherical()

transform1

[Translate(shift:(-483.898, -523.095)),
 Scale(factor:0.0025846471956577927),
 Expand(inv:False, thr:0),
 Rotate(axis:(-0.2051122436882468, -0.049549079923125856, -0.9774834301244932), angle:0.03851808963765877),
 ToSpherical(inv:False)]

In [11]:
view2 = View.from_header(header2)
transform2 = view2.to_spherical()

transform2

[Translate(shift:(-419.914, -463.457)),
 Scale(factor:0.0026878830233308245),
 Expand(inv:False, thr:0),
 Rotate(axis:(-0.2975432814811417, -0.4626568790166745, 0.8351147274141346), angle:0.09273301118805073),
 ToSpherical(inv:False)]

Calculating the coordinate transformation from one dataset to another

In [12]:
transform = transform1 - transform2

transform

[Translate(shift:(-483.898, -523.095)),
 Scale(factor:0.0025846471956577927),
 Expand(inv:False, thr:0),
 Rotate(axis:(0.026792949842642577, 0.4623489576463984, 0.8862931677509528), angle:0.08206265406373421),
 Expand(inv:True, thr:0),
 Scale(factor:372.04),
 Translate(shift:(419.914, 463.457))]

Reprojecting the second dataset onto the first one and showing the result

In [13]:
data2_ = reproject(data2, header2, header1, correct_mu=True, mu_thr=0.)

In [14]:
show_data(data2_, header1)

In [20]:
show_data(data2_ - data1, header1)